In [12]:
# pip install python-dotenv

In [14]:
import requests
import numpy as np
from typing import List, Dict, Optional, Union

class QdrantManager:
    def __init__(self, url: str, api_key: str):
        """
        Initialize QdrantManager with connection details.
        
        Args:
            url: Qdrant server URL
            api_key: API key for authentication
        """
        self.url = url.rstrip('/')
        self.headers = {
            "api-key": api_key,
            "Content-Type": "application/json"
        }

    def _make_request(self, method: str, endpoint: str, data: Optional[Dict] = None) -> Dict:
        """
        Make HTTP request to Qdrant API.
        
        Args:
            method: HTTP method (GET, POST, PUT, DELETE)
            endpoint: API endpoint
            data: Request data (optional)
        """
        url = f"{self.url}/{endpoint.lstrip('/')}"
        try:
            response = requests.request(method, url, headers=self.headers, json=data)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error making request: {e}")
            raise

    def list_collections(self) -> List[str]:
        """List all available collections."""
        response = self._make_request('GET', 'collections')
        collections = response['result']['collections']
        print("\nAvailable Collections:")
        for collection in collections:
            print(f"- {collection['name']}")
        return [c['name'] for c in collections]

    def collection_exists(self, name: str) -> bool:
        """Check if a collection exists quietly without error messages."""
        try:
            self._make_request('GET', f'collections/{name}')
            return True
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 404:
                return False
            raise
        except requests.exceptions.RequestException:
            raise

    def create_collection(self, name: str, vector_size: int = 1536, distance: str = "Cosine", overwrite: bool = False) -> bool:
        """
        Create a new collection.
        
        Args:
            name: Collection name
            vector_size: Dimension of vectors (default 1536 for OpenAI embeddings)
            distance: Distance metric (Cosine, Euclid, Dot)
            overwrite: If True, will delete existing collection with same name
        """
        try:
            if self.collection_exists(name):
                if overwrite:
                    print(f"Collection '{name}' already exists. Deleting...")
                    self.delete_collection(name)
                else:
                    print(f"Collection '{name}' already exists. Set overwrite=True to replace it.")
                    return False

            data = {
                "vectors": {
                    "size": vector_size,
                    "distance": distance
                }
            }
            response = self._make_request('PUT', f'collections/{name}', data)
            success = response['result']
            if success:
                print(f"Collection '{name}' created successfully")
            return success
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 409:
                print(f"Collection '{name}' already exists. Set overwrite=True to replace it.")
            raise

    def delete_collection(self, name: str) -> bool:
        """Delete a collection."""
        response = self._make_request('DELETE', f'collections/{name}')
        success = response['result']
        if success:
            print(f"Collection '{name}' deleted successfully")
        return success

    def get_collection_info(self, name: str) -> Dict:
        """Get detailed information about a collection."""
        response = self._make_request('GET', f'collections/{name}')
        info = response['result']
        print(f"\nCollection '{name}' info:")
        print(f"- Total points: {info['points_count']}")
        print(f"- Vector size: {info['config']['params']['vectors']['size']}")
        print(f"- Distance metric: {info['config']['params']['vectors']['distance']}")
        return info

    def insert_points(self, collection_name: str, vectors: np.ndarray, 
                     payloads: Optional[List[Dict]] = None, batch_size: int = 100) -> bool:
        """
        Insert points into collection.
        
        Args:
            collection_name: Target collection
            vectors: Numpy array of vectors
            payloads: List of payload dictionaries (optional)
            batch_size: Number of vectors to insert in each batch
        """
        if payloads and len(vectors) != len(payloads):
            raise ValueError("Number of vectors and payloads must match")

        for i in range(0, len(vectors), batch_size):
            batch_vectors = vectors[i:i + batch_size]
            batch_payloads = payloads[i:i + batch_size] if payloads else None

            points = []
            for j, vector in enumerate(batch_vectors):
                point = {
                    "id": i + j,
                    "vector": vector.tolist()
                }
                if batch_payloads:
                    point["payload"] = batch_payloads[j]
                points.append(point)

            data = {"points": points}
            response = self._make_request('PUT', f'collections/{collection_name}/points', data)
            
            if not response['result']:
                return False
            
        print(f"Successfully inserted {len(vectors)} points")
        return True

    def delete_points(self, collection_name: str, point_ids: List[int]) -> bool:
        """Delete points by their IDs."""
        data = {"points": point_ids}
        response = self._make_request('POST', f'collections/{collection_name}/points/delete', data)
        success = response['result']
        if success:
            print(f"Successfully deleted {len(point_ids)} points")
        return success

    def search_points(self, collection_name: str, query_vector: np.ndarray, 
                     limit: int = 5, filter_params: Optional[Dict] = None, 
                     score_threshold: Optional[float] = None) -> List[Dict]:
        """
        Search for similar vectors.
        
        Args:
            collection_name: Target collection
            query_vector: Vector to search for
            limit: Maximum number of results
            filter_params: Filter conditions
            score_threshold: Minimum similarity score
        """
        data = {
            "vector": query_vector.tolist(),
            "limit": limit,
            "with_payload": True
        }
        
        if filter_params:
            data["filter"] = filter_params
        if score_threshold:
            data["score_threshold"] = score_threshold

        response = self._make_request('POST', f'collections/{collection_name}/points/search', data)
        results = response['result']

        print("\nSearch Results:")
        for idx, match in enumerate(results):
            print(f"\nMatch {idx + 1}:")
            print(f"- ID: {match['id']}")
            print(f"- Score: {match['score']:.4f}")
            if 'payload' in match:
                print(f"- Payload: {match['payload']}")

        return results

    def get_points(self, collection_name: str, limit: int = 10, 
                  offset: Optional[Union[str, Dict]] = None) -> Dict:
        """
        Get points from collection with pagination.
        
        Args:
            collection_name: Target collection
            limit: Number of points to return
            offset: Pagination offset
        """
        data = {
            "limit": limit,
            "with_payload": True,
            "with_vector": True
        }
        if offset:
            data["offset"] = offset

        response = self._make_request('POST', f'collections/{collection_name}/points/scroll', data)
        points = response['result']

        print(f"\nRetrieved {len(points['points'])} points:")
        for point in points['points']:
            print(f"\nPoint ID: {point['id']}")
            if 'payload' in point:
                print(f"Payload: {point['payload']}")

        return points

    def list_all_collections_info(self) -> List[Dict]:
        """
        Get detailed information about all collections including their point counts.
        Returns a list of dictionaries containing collection details.
        """
        collections_info = []
        collections = self.list_collections()
        
        print("\nDetailed Collections Information:")
        print("-" * 50)
        
        for collection_name in collections:
            info = self.get_collection_info(collection_name)
            collections_info.append({
                "name": collection_name,
                "points_count": info["points_count"],
                "vector_size": info["config"]["params"]["vectors"]["size"],
                "distance": info["config"]["params"]["vectors"]["distance"]
            })
            print(f"\nCollection: {collection_name}")
            print(f"- Total points: {info['points_count']}")
            print(f"- Vector size: {info['config']['params']['vectors']['size']}")
            print(f"- Distance metric: {info['config']['params']['vectors']['distance']}")
            print("-" * 50)
        
        return collections_info

    def delete_test_collections(self, test_prefix: str = "test") -> List[str]:
        """
        Delete all collections that start with the test prefix.
        
        Args:
            test_prefix: Prefix to identify test collections (default: "test")
            
        Returns:
            List of deleted collection names
        """
        deleted_collections = []
        collections = self.list_collections()
        
        for collection_name in collections:
            if collection_name.lower().startswith(test_prefix.lower()):
                if self.delete_collection(collection_name):
                    deleted_collections.append(collection_name)
        
        if deleted_collections:
            print(f"\nDeleted {len(deleted_collections)} test collections: {', '.join(deleted_collections)}")
        else:
            print(f"\nNo test collections found with prefix '{test_prefix}'")
            
        return deleted_collections
    
# Example usage:
if __name__ == "__main__":
    from dotenv import load_dotenv
    import os

    # Load environment variables from .env file
    load_dotenv()
    # Initialize manager
    url = os.getenv("QDRANT_URL", "http://localhost:6333")
    api_key= os.getenv("QDRANT_API_KEY", "default")
    qdrant = QdrantManager(
        url=url,
        api_key=api_key
    )
    
    # List collections
    collections = qdrant.list_collections()
    
    # Create a new collection
    qdrant.create_collection("test_collection")
    
    # Insert some test points
    vectors = np.random.rand(5, 1536)  # 5 random vectors
    payloads = [
        {"name": f"doc_{i}", "category": "test"}
        for i in range(5)
    ]
    qdrant.insert_points("test_collection", vectors, payloads)
    
    # Search
    query = np.random.rand(1536)
    results = qdrant.search_points(
        "test_collection", 
        query,
        limit=3,
        filter_params={
            "must": [
                {"key": "category", "match": {"value": "test"}}
            ]
        }
    )


Available Collections:
- my_docs
- test_collection
Collection 'test_collection' already exists. Set overwrite=True to replace it.
Successfully inserted 5 points

Search Results:

Match 1:
- ID: 3
- Score: 0.7503
- Payload: {'category': 'test', 'name': 'doc_3'}

Match 2:
- ID: 2
- Score: 0.7495
- Payload: {'name': 'doc_2', 'category': 'test'}

Match 3:
- ID: 4
- Score: 0.7482
- Payload: {'name': 'doc_4', 'category': 'test'}


In [15]:
# See all collections and their details
collections_info = qdrant.list_all_collections_info()


Available Collections:
- my_docs
- test_collection

Detailed Collections Information:
--------------------------------------------------

Collection 'my_docs' info:
- Total points: 3
- Vector size: 1536
- Distance metric: Cosine

Collection: my_docs
- Total points: 3
- Vector size: 1536
- Distance metric: Cosine
--------------------------------------------------

Collection 'test_collection' info:
- Total points: 5
- Vector size: 1536
- Distance metric: Cosine

Collection: test_collection
- Total points: 5
- Vector size: 1536
- Distance metric: Cosine
--------------------------------------------------


In [16]:
qdrant.get_points("test_collection", limit=3)


Retrieved 3 points:

Point ID: 0
Payload: {'name': 'doc_0', 'category': 'test'}

Point ID: 1
Payload: {'category': 'test', 'name': 'doc_1'}

Point ID: 2
Payload: {'name': 'doc_2', 'category': 'test'}


{'points': [{'id': 0,
   'payload': {'name': 'doc_0', 'category': 'test'},
   'vector': [0.0036741558,
    0.028402148,
    0.029181361,
    0.0039151395,
    0.0028152268,
    0.0061633415,
    0.003465624,
    0.041346613,
    0.03920328,
    0.01025315,
    0.025445776,
    0.021156188,
    0.03379905,
    0.011011039,
    0.038849633,
    0.02629795,
    0.009281391,
    0.011445485,
    0.008143078,
    0.033931065,
    0.0038472167,
    0.03853765,
    0.04406346,
    0.033459015,
    0.024992082,
    0.026935535,
    0.040247984,
    0.035201076,
    0.013291407,
    0.023032501,
    0.033430938,
    0.008626228,
    0.035009302,
    0.043117024,
    0.024133744,
    0.03440997,
    0.018345976,
    0.009638314,
    0.033545095,
    0.024875825,
    0.03401043,
    0.027298246,
    0.015329023,
    0.014514194,
    0.021394942,
    0.025447335,
    0.030912265,
    0.002876419,
    0.000100855854,
    0.024235277,
    0.013676339,
    0.013138842,
    0.011239152,
    0.00710302

In [17]:
# Delete all test collections (collections starting with "test")
deleted = qdrant.delete_test_collections()


Available Collections:
- my_docs
- test_collection
Collection 'test_collection' deleted successfully

Deleted 1 test collections: test_collection


In [18]:
# Or delete collections with a different prefix
deleted = qdrant.delete_test_collections(test_prefix="demo_")


Available Collections:
- my_docs

No test collections found with prefix 'demo_'


In [19]:
# Check collections after deletion
collections_info = qdrant.list_all_collections_info()


Available Collections:
- my_docs

Detailed Collections Information:
--------------------------------------------------

Collection 'my_docs' info:
- Total points: 3
- Vector size: 1536
- Distance metric: Cosine

Collection: my_docs
- Total points: 3
- Vector size: 1536
- Distance metric: Cosine
--------------------------------------------------
